In [15]:

# BOILER PLATE, MUST BE RUN ON SUBMIT NODE
%load_ext autoreload
%autoreload 2
import socket
print("Running on host: ", socket.gethostname())

import sys 
lib_path = '/fsx_0/user/tranx/experiments'
if lib_path not in sys.path:
    sys.path.append(lib_path)
#=================================================
    
import plotly.express as px
import plotly.graph_objects as go
import re
import time
import shutil
import json
import glob
import pandas as pd
import os
import glob
from pprint import pprint
from lib import eval_helper
from lib import utils
from lib import slurm
from lib.slurm import run_sbatch_job

EVAL_CONFIG_OVERWRITE = "/fsx_0/user/tranx/rsync/llm_mm_aligner/experiments/aws/mm9_sft/eval_overwrite"

ev = eval_helper.EvalHelper(
    code_dir="/fsx_0/user/tranx/rsync",
    eval_sbatch="/fsx_0/user/tranx/rsync/llm_mm_aligner/experiments/aws/launch_eval_sbatch.sh",
    eval_config_dir="/fsx_0/user/tranx/rsync/llm_mm_aligner/experiments/aws/mm9_sft/eval_overwrite"
)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Running on host:  submit-1


# Create eval config

In [10]:
data_files = {}
for f in glob.glob("/fsx_0/user/tranx/adel_prod/llm_mm_aligner/experiments/aws_adel/eval_sft_babyLora_emb65/eval_*.json"):
    conf = utils.read_json(f)
    bm = f.split("/")[-1].replace("eval_", "").replace(".json", "")
    # print(bm)
    data_files[bm] = {
        "train_file": conf["eval_args"]["train_file"],
        "validation_file": conf["eval_args"]["validation_file"]
    }

data_files

{'infographics_w_ocr': {'train_file': '/fsx_0/dataset01/infographicsvqa/processed_val.jsonl',
  'validation_file': '/fsx_0/dataset01/infographicsvqa/processed_val.jsonl'},
 'infographics': {'train_file': '/fsx_0/dataset01/infographicsvqa/processed_val.jsonl',
  'validation_file': '/fsx_0/dataset01/infographicsvqa/processed_val.jsonl'},
 'mmbench': {'train_file': '/fsx_0/dataset01/mmbench/processed_dev_20231212.json',
  'validation_file': '/fsx_0/dataset01/mmbench/processed_dev_20231212.json'},
 'mathvista': {'train_file': '/fsx_0/dataset01/mathvista/test.jsonl',
  'validation_file': '/fsx_0/dataset01/mathvista/test.jsonl'},
 'ai2d': {'train_file': '/fsx_0/dataset01/ai2d/ai2d_test.jsonl',
  'validation_file': '/fsx_0/dataset01/ai2d/ai2d_test.jsonl'},
 'docvqa': {'train_file': '/fsx_0/dataset01/docvqa/val.jsonl',
  'validation_file': '/fsx_0/dataset01/docvqa/val.jsonl'},
 'textvqa': {'train_file': '/fsx_0/dataset01/textvqa/text_vqa_val_set_updated.jsonl',
  'validation_file': '/fsx_0/dat

In [14]:
original_dir = "/fsx_0/user/tranx/experiments/eval/sft_eval_overwrite_fb"
new_dir = "/fsx_0/user/tranx/rsync/llm_mm_aligner/experiments/aws/mm9_sft/eval_overwrite"

for file in glob.glob(f"{original_dir}/*.json"):
    benchmark = os.path.basename(file).replace(".json", "")
    config = utils.read_json(file)
    
    for k in ["scheduler_type", "num_gpus", "num_nodes"]:
        config.pop(k)
        
    config["eval_args"].update({
        "output_dir": "OUTPUT_DIR",
        "resume_from_checkpoint": "CHECKPOINT_PATH",
        "logging_dir": "LOGGING_DIR",
        "tb_logdir": "TB_LOGDIR",
        "eval_ckpt": "EVAL_CHECKPOINT",
        "train_file": data_files[benchmark]["train_file"],
        "validation_file": data_files[benchmark]["validation_file"]
    })
        
    print(benchmark)
    new_file = f"{new_dir}/eval_{benchmark}.json"
    utils.save_json(config, new_file)
    
    # break

chartqa
textvqa
mmbench
infographics_w_ocr
infographics
mmmu
mathvista
ai2d
docvqa
vqa


# New - Flywheel

In [5]:
ALIGNER_RSYNC = "/fsx_0/user/tranx/rsync"
EVAL_SBATCH = "/fsx_0/user/tranx/rsync/llm_mm_aligner/experiments/aws/launch_eval_sbatch.sh"
EVAL_CONFIG_OVERWRITE = "/fsx_0/user/tranx/rsync/llm_mm_aligner/experiments/aws/mm9_sft/eval_overwrite"
EVAL_CONFIG_OVERWRITE_v5 = "/fsx_0/user/tranx/rsync/llm_mm_aligner/experiments/aws/mm9_sft/eval_overwrite_v5"


OUTPUT_SFT8 = "/fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_flywheel"
OUTPUT_SFT9 = "/fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_r26a_flywheel"
OUTPUT_SFT10 = "/fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_flywheel_v5"
OUTPUT_SFT11 = "/fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_r26a_flywheel_v5"

In [40]:
eval_helper.run_eval_sweep(
    output_dir=OUTPUT_SFT8,
    eval_sbatch=EVAL_SBATCH,
    eval_config_dir=EVAL_CONFIG_OVERWRITE,
    aligner_parent_dir=ALIGNER_RSYNC,
    # benchmarks=["mmmu"],
    eval_plan="eval_fix",
    checkpoint_interval=180 #100
)

New checkpoints: [180]
Start eval for /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_flywheel/checkpoint-180
eval jobs saved to: /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_flywheel/eval_fix/eval_jobs_checkpoint-180.json


In [5]:
eval_helper.get_eval_scores_all(OUTPUT_SFT8, eval_plan="eval_fix")

,mmmu_v2,mmmu_v1,docvqa,mathvista,ai2d,chartqa,vqa,textvqa,infographics_w_ocr,infographics,mmbench
100,0.5922,0.5833,0.7866,0.543,0.7979,0.5912,0.7525,0.71948,0.7223,0.6822,0.771
180,0.5856,0.5833,0.7966,0.554,0.784,0.5996,0.7497,0.72328,0.7296,0.6881,0.7784


In [66]:
eval_helper.run_eval_sweep(
    output_dir=OUTPUT_SFT9,
    eval_sbatch=EVAL_SBATCH,
    eval_config_dir=EVAL_CONFIG_OVERWRITE,
    aligner_parent_dir=ALIGNER_RSYNC,
    # benchmarks=["mmmu"],
    eval_plan="eval_fix",
    checkpoint_interval=100
)

New checkpoints: [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000, 1100, 1200, 1300, 1400, 1500, 1600, 1700, 1800, 1900, 2000, 2100, 2200, 2300, 2400, 2500, 2600, 2700, 2800, 2900, 3000, 3100, 3200, 3300, 3400, 3500, 3600, 3700, 3800, 3900, 4000, 4100, 4200, 4300, 4400, 4500, 4600, 4700, 4800, 4900, 5000, 5100, 5200]
Start eval for /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_r26a_flywheel/checkpoint-100
eval jobs saved to: /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_r26a_flywheel/eval_fix/eval_jobs_checkpoint-100.json
Start eval for /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_r26a_flywheel/checkpoint-200
eval jobs saved to: /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_r26a_flywheel/eval_fix/eval_jobs_checkpoint-200.json
Start eval for /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_r26a_flywheel/checkpoint-300
eval jobs saved to: /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_r26a_flyw

In [25]:
eval_helper.get_eval_scores_all(OUTPUT_SFT9, eval_plan="eval_fix")

,mmmu_v2,mmmu_v1,docvqa,mathvista,ai2d,chartqa,vqa,textvqa,infographics_w_ocr,infographics,mmbench
100,0.54,0.5256,0.751,0.56,0.8174,0.6404,0.4491,0.5057,0.6991,0.6439,0.7661
200,0.5322,0.5178,0.7321,0.565,0.8067,0.617,0.4697,0.58896,0.6828,0.626,0.7703
300,0.5422,0.5267,0.6963,0.561,0.807,0.5897,0.5258,0.6329,0.6529,0.5986,0.7725
400,0.5444,0.5289,0.6801,0.56,0.8041,0.5885,0.5619,0.65144,0.6389,0.581,0.7799
500,0.5378,0.5211,0.665,0.562,0.8025,0.5814,0.5233,0.64404,0.6341,0.5717,0.7775
600,0.54,0.5278,0.6601,0.569,0.8041,0.5762,0.5223,0.64296,0.6356,0.5711,0.771
700,0.5311,0.5178,0.6529,0.569,0.8038,0.5726,0.5095,0.63926,0.6325,0.569,0.7731
800,0.5356,0.52,0.6537,0.569,0.8067,0.5716,0.4881,0.65026,0.6271,0.5695,0.7707
900,0.5278,0.5167,0.6625,0.569,0.8028,0.575,0.5169,0.656,0.6347,0.5684,0.7723
1000,0.5456,0.5244,0.6652,0.575,0.8018,0.5755,0.5012,0.65088,0.6318,0.5711,0.7757


In [67]:
eval_helper.run_eval_sweep(
    output_dir=OUTPUT_SFT10,
    eval_sbatch=EVAL_SBATCH,
    eval_config_dir=EVAL_CONFIG_OVERWRITE,
    aligner_parent_dir=ALIGNER_RSYNC,
    # benchmarks=["mmmu"],
    eval_plan="eval_fix",
    checkpoint_interval=100
)

New checkpoints: [100]
Start eval for /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_flywheel_v5/checkpoint-100
eval jobs saved to: /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_flywheel_v5/eval_fix/eval_jobs_checkpoint-100.json


In [8]:
eval_helper.run_eval_sweep(
    output_dir=OUTPUT_SFT10,
    eval_sbatch=EVAL_SBATCH,
    eval_config_dir=EVAL_CONFIG_OVERWRITE_v5,
    aligner_parent_dir=ALIGNER_RSYNC,
    benchmarks=["mmmu"],
    eval_plan="eval_fix_v5",
    checkpoint_interval=100
)

New checkpoints: [100]
Start eval for /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_flywheel_v5/checkpoint-100
eval jobs saved to: /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_flywheel_v5/eval_fix_v5/eval_jobs_checkpoint-100.json


In [11]:
eval_helper.run_eval_sweep(
    output_dir=OUTPUT_SFT11,
    eval_sbatch=EVAL_SBATCH,
    eval_config_dir=EVAL_CONFIG_OVERWRITE_v5,
    aligner_parent_dir=ALIGNER_RSYNC,
    benchmarks=["mmmu"],
    eval_plan="eval_fix_v5",
    checkpoint_interval=100
)

New checkpoints: [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000, 1100]
Start eval for /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_r26a_flywheel_v5/checkpoint-100
eval jobs saved to: /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_r26a_flywheel_v5/eval_fix_v5/eval_jobs_checkpoint-100.json
Start eval for /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_r26a_flywheel_v5/checkpoint-200
eval jobs saved to: /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_r26a_flywheel_v5/eval_fix_v5/eval_jobs_checkpoint-200.json
Start eval for /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_r26a_flywheel_v5/checkpoint-300
eval jobs saved to: /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_r26a_flywheel_v5/eval_fix_v5/eval_jobs_checkpoint-300.json
Start eval for /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_r26a_flywheel_v5/checkpoint-400
eval jobs saved to: /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336p

In [23]:
eval_helper.get_eval_scores_all(OUTPUT_SFT8, eval_plan="eval_fix")

,mmmu_v2,mmmu_v1,docvqa,mathvista,ai2d,chartqa,vqa,textvqa,infographics_w_ocr,infographics,mmbench
100,0.5922,0.5833,0.7866,0.543,0.7979,0.5912,0.7525,0.71948,0.7223,0.6822,0.771


In [24]:
eval_helper.get_eval_scores_all(OUTPUT_SFT10, eval_plan="eval_fix")

,mmmu_v2,mmmu_v1,docvqa,mathvista,ai2d,chartqa,vqa,textvqa,infographics_w_ocr,infographics,mmbench
100,NaN,NaN,0.7558,0.54,NaN,0.5546,NaN,NaN,NaN,NaN,0.7739


In [14]:
eval_helper.get_eval_scores_all(OUTPUT_SFT10, eval_plan="eval_fix_v5")

,mmmu_v2,mmmu_v1,docvqa,mathvista,ai2d,chartqa,vqa,textvqa,infographics_w_ocr,infographics,mmbench


In [15]:
eval_helper.get_eval_scores_all(OUTPUT_SFT11, eval_plan="eval_fix_v5")

,mmmu_v2,mmmu_v1,docvqa,mathvista,ai2d,chartqa,vqa,textvqa,infographics_w_ocr,infographics,mmbench


# Test

In [43]:
EVAL_CONFIG_OVERWRITE

'/fsx_0/user/tranx/rsync/llm_mm_aligner/experiments/aws/mm9_sft/eval_overwrite'

In [44]:
EVAL_SBATCH

'/fsx_0/user/tranx/rsync/llm_mm_aligner/experiments/aws/launch_eval_sbash.sh'

In [46]:
eval_helper.run_eval_sweep(
    output_dir=OUTPUT_SFT8,
    eval_sbatch=EVAL_SBATCH,
    eval_config_dir=EVAL_CONFIG_OVERWRITE,
    aligner_parent_dir=ALIGNER_RSYNC,
    benchmarks=["mmmu"],
    eval_plan="eval_test",
    checkpoint_interval=160, #100,
    print_cmd=True
)

New checkpoints: [160]
Start eval for /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_flywheel/checkpoint-160
sbatch --parsable --job-name=eval_mmmu /fsx_0/user/tranx/rsync/llm_mm_aligner/experiments/aws/launch_eval_sbash.sh /fsx_0/user/tranx/rsync /fsx_0/user/tranx/rsync/llm_mm_aligner/experiments/aws/mm9_sft/eval_overwrite/eval_mmmu.json /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_flywheel mmmu 160
eval jobs saved to: /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_flywheel/eval_test/eval_jobs_checkpoint-160.json


# SFT 1, 3, 4

In [8]:
OUTPUT_SFT1 = "/fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_5800_BabyLora_16nodes"
OUTPUT_SFT3 = "/fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_r17e"
OUTPUT_SFT4 = "/fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_r30c"

In [6]:
eval_helper.run_eval_sweep(
    output_dir=OUTPUT_SFT1,
    eval_sbatch=EVAL_SBATCH,
    eval_config_dir=EVAL_CONFIG_OVERWRITE,
    aligner_parent_dir=ALIGNER_RSYNC,
    eval_plan="eval_fix",
    checkpoints=[2200],
    update_if_exists=True
)

New checkpoints: [2200]
Start eval for /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_5800_BabyLora_16nodes/checkpoint-2200
Command '['sbatch', '--parsable', '--job-name=eval_mmmu', '/fsx_0/user/tranx/rsync/llm_mm_aligner/experiments/aws/launch_eval_sbash.sh', '/fsx_0/user/tranx/rsync', '/fsx_0/user/tranx/rsync/llm_mm_aligner/experiments/aws/mm9_sft/eval_overwrite/eval_mmmu.json', '/fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_5800_BabyLora_16nodes', 'mmmu', '2200']' returned non-zero exit status 1.


sbatch: error: Unable to open file /fsx_0/user/tranx/rsync/llm_mm_aligner/experiments/aws/launch_eval_sbash.sh


TypeError: int() argument must be a string, a bytes-like object or a real number, not 'NoneType'

In [9]:
ev.get_scores(OUTPUT_SFT1, [2200], eval_plan="eval_fix")

,mmmu_v2,mmmu_v1,docvqa,mathvista,ai2d,chartqa,vqa,textvqa,infographics_w_ocr,infographics,mmbench
2200,0.5578,0.5411,0.6981,0.507,0.8394,0.5643,0.7041,0.72188,0.5382,0.4622,0.7679


In [34]:
eval_helper.run_eval_sweep(
    output_dir=OUTPUT_SFT3,
    eval_sbatch=EVAL_SBATCH,
    eval_config_dir=EVAL_CONFIG_OVERWRITE,
    aligner_parent_dir=ALIGNER_RSYNC,
    eval_plan="eval_fix",
    checkpoints=[1100],
    update_if_exists=True
)

New checkpoints: [1100]
Start eval for /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_r17e/checkpoint-1100
eval jobs saved to: /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_r17e/eval_fix/eval_jobs_checkpoint-1100.json


In [8]:
eval_helper.get_eval_scores_all(OUTPUT_SFT3, [1100], eval_plan="eval_fix")

,mmmu_v2,mmmu_v1,docvqa,mathvista,ai2d,chartqa,vqa,textvqa,infographics_w_ocr,infographics,mmbench
1100,0.5411,0.5289,0.6863,0.578,0.8157,0.5834,0.5938,0.65978,0.6559,0.5903,0.7846


In [35]:
eval_helper.run_eval_sweep(
    output_dir=OUTPUT_SFT4,
    eval_sbatch=EVAL_SBATCH,
    eval_config_dir=EVAL_CONFIG_OVERWRITE,
    aligner_parent_dir=ALIGNER_RSYNC,
    eval_plan="eval_fix",
    checkpoints=[900],
    update_if_exists=True
)

New checkpoints: [900]
Start eval for /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_r30c/checkpoint-900
eval jobs saved to: /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_r30c/eval_fix/eval_jobs_checkpoint-900.json


In [9]:
eval_helper.get_eval_scores_all(OUTPUT_SFT4, [900], eval_plan="eval_fix")

,mmmu_v2,mmmu_v1,docvqa,mathvista,ai2d,chartqa,vqa,textvqa,infographics_w_ocr,infographics,mmbench
900,0.5267,0.5178,0.6674,0.569,0.8044,0.5847,0.6682,0.66334,0.6473,0.5746,0.7825


# Stage 2

In [1]:
OUTPUT_S2_5800 = "/fsx_0/checkpoints/aligner/mm9_stage2/MH19_336px_128nodes_exp28"
OUTPUT_S2_8800 = "/fsx_0/checkpoints/tranx/MM9-Stage2-70B/MH19_336px_128nodes_exp30"
OUTPUT_S2_31b = "/fsx_0/checkpoints/ahmadyan/MM9-Stage2-70B/MH19_336px_exp31b"

In [13]:
eval_helper.run_eval_sweep(
    output_dir=OUTPUT_S2_5800,
    eval_sbatch=EVAL_SBATCH,
    eval_config_dir=EVAL_CONFIG_OVERWRITE,
    aligner_parent_dir=ALIGNER_RSYNC,
    eval_plan="eval_fix",
    checkpoints=[5800],
    update_if_exists=True
)

New checkpoints: [5800]
Start eval for /fsx_0/checkpoints/aligner/mm9_stage2/MH19_336px_128nodes_exp28/checkpoint-5800
eval jobs saved to: /fsx_0/checkpoints/aligner/mm9_stage2/MH19_336px_128nodes_exp28/eval_fix/eval_jobs_checkpoint-5800.json


In [14]:
eval_helper.run_eval_sweep(
    output_dir=OUTPUT_S2_8800,
    eval_sbatch=EVAL_SBATCH,
    eval_config_dir=EVAL_CONFIG_OVERWRITE,
    aligner_parent_dir=ALIGNER_RSYNC,
    eval_plan="eval_fix",
    checkpoints=[8800],
    update_if_exists=True
)

New checkpoints: [8800]
Start eval for /fsx_0/checkpoints/tranx/MM9-Stage2-70B/MH19_336px_128nodes_exp30/checkpoint-8800
eval jobs saved to: /fsx_0/checkpoints/tranx/MM9-Stage2-70B/MH19_336px_128nodes_exp30/eval_fix/eval_jobs_checkpoint-8800.json


In [14]:
ev.run_eval_sweep(OUTPUT_S2_31b, eval_plan="evals_harry")

New checkpoints: [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]
Start eval for /fsx_0/checkpoints/ahmadyan/MM9-Stage2-70B/MH19_336px_exp31b/checkpoint-200
eval jobs saved to: /fsx_0/checkpoints/ahmadyan/MM9-Stage2-70B/MH19_336px_exp31b/evals_harry/eval_jobs_checkpoint-200.json
Start eval for /fsx_0/checkpoints/ahmadyan/MM9-Stage2-70B/MH19_336px_exp31b/checkpoint-400
eval jobs saved to: /fsx_0/checkpoints/ahmadyan/MM9-Stage2-70B/MH19_336px_exp31b/evals_harry/eval_jobs_checkpoint-400.json
Start eval for /fsx_0/checkpoints/ahmadyan/MM9-Stage2-70B/MH19_336px_exp31b/checkpoint-600
eval jobs saved to: /fsx_0/checkpoints/ahmadyan/MM9-Stage2-70B/MH19_336px_exp31b/evals_harry/eval_jobs_checkpoint-600.json
Start eval for /fsx_0/checkpoints/ahmadyan/MM9-Stage2-70B/MH19_336px_exp31b/checkpoint-800
eval jobs saved to: /fsx_0/checkpoints/ahmadyan/MM9-Stage2-70B/MH19_336px_exp31b/evals_harry/eval_jobs_checkpoint-800.json
Start eval for /fsx_0/checkpoints/ahmadyan/MM9-Stage2-70B/MH19_336px_e

# Flywheel

In [6]:
EVAL_CONFIG_DIR_EMB65_v5

'/fsx_0/user/tranx/adel_prod/llm_mm_aligner/experiments/aws_adel/eval_sft_babyLora_emb65_v5'

In [4]:
ALIGNER_ADEL_PROD = '/fsx_0/user/tranx/adel_prod'
ALIGNER_ADEL_PROD_FSDP = '/fsx_0/user/tranx/adel_prod_fsdp'
EVAL_SBATCH = "/fsx_0/user/tranx/experiments/eval/sbash_eval.sh"
EVAL_CONFIG_DIR_EMB65 = f"{ALIGNER_ADEL_PROD}/llm_mm_aligner/experiments/aws_adel/eval_sft_babyLora_emb65"
EVAL_CONFIG_DIR_EMB65_v5 = f"{ALIGNER_ADEL_PROD}/llm_mm_aligner/experiments/aws_adel/eval_sft_babyLora_emb65_v5"

OUTPUT_SFT8 = "/fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_flywheel"
OUTPUT_SFT9 = "/fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_r26a_flywheel"
OUTPUT_SFT10 = "/fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_flywheel_v5"

# BENCHMARKS = ["mathvista", "vqa"]
BENCHMARKS = ["mmmu"]

In [7]:
# train v4 - eval v4
eval_helper.run_eval_sweep(
    output_dir=OUTPUT_SFT8,
    eval_sbatch=EVAL_SBATCH,
    eval_config_dir=EVAL_CONFIG_DIR_EMB65,
    aligner_parent_dir=ALIGNER_ADEL_PROD_FSDP,
    benchmarks=["mmmu"],
    eval_plan="eval_v4",
    checkpoint_interval=100
)

New checkpoints: [100]
Start eval for /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_flywheel/checkpoint-100
eval jobs saved to: /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_flywheel/eval_v4/eval_jobs_checkpoint-100.json


In [37]:
eval_helper.get_eval_scores_all(OUTPUT_SFT8, eval_plan="eval_v4")

,mmmu_v2,mmmu_v1,docvqa,mathvista,ai2d,chartqa,vqa,textvqa,infographics_w_ocr,infographics,mmbench


In [19]:
# train v4 - eval v5
eval_helper.run_eval_sweep(
    output_dir=OUTPUT_SFT8,
    eval_sbatch=EVAL_SBATCH,
    eval_config_dir=EVAL_CONFIG_DIR_EMB65_v5,
    aligner_parent_dir=ALIGNER_ADEL_PROD_FSDP,
    benchmarks=["mmmu"],
    eval_plan="eval_v5_2",
    checkpoint_interval=100
)

New checkpoints: [100]
Start eval for /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_flywheel/checkpoint-100
eval jobs saved to: /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_flywheel/eval_v5_2/eval_jobs_checkpoint-100.json


In [36]:
eval_helper.get_eval_scores_all(OUTPUT_SFT9, eval_plan="eval_v5")

,mmmu_v2,mmmu_v1,docvqa,mathvista,ai2d,chartqa,vqa,textvqa,infographics_w_ocr,infographics,mmbench


In [8]:
# train v5 - eval v4
eval_helper.run_eval_sweep(
    output_dir=OUTPUT_SFT10,
    eval_sbatch=EVAL_SBATCH,
    eval_config_dir=EVAL_CONFIG_DIR_EMB65,
    aligner_parent_dir=ALIGNER_ADEL_PROD_FSDP,
    benchmarks=["mmmu"],
    eval_plan="eval_v4",
    checkpoint_interval=100
)

New checkpoints: [100]
Start eval for /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_flywheel_v5/checkpoint-100
eval jobs saved to: /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_flywheel_v5/eval_v4/eval_jobs_checkpoint-100.json


In [24]:
eval_helper.get_eval_jobs_record("/fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_r26a",
                                 100)

'/fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_r26a/eval/eval_jobs_checkpoint-100.json'

In [27]:
utils.read_json('/fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_r26a/evals/eval_jobs_checkpoint-200.json')

OrderedDict([('mmmu', OrderedDict([('200', 21877)])),
             ('docvqa', OrderedDict([('200', 21878)])),
             ('mathvista', OrderedDict([('200', 21879)])),
             ('ai2d', OrderedDict([('200', 21880)])),
             ('chartqa', OrderedDict([('200', 21881)])),
             ('vqa', OrderedDict([('200', 21882)])),
             ('textvqa', OrderedDict([('200', 21883)])),
             ('infographics_w_ocr', OrderedDict([('200', 21884)])),
             ('infographics', OrderedDict([('200', 21885)])),
             ('mmbench', OrderedDict([('200', 21886)]))])

In [29]:
utils.read_json("/fsx_0/checkpoints/tranx/MM9-Stage2-70B/MH19_336px_128nodes_exp28/evals/eval_jobs_checkpoint-200.json")

OrderedDict([('mmmu', OrderedDict([('200', 14227)])),
             ('docvqa', OrderedDict([('200', 14228)])),
             ('mathvista', OrderedDict([('200', 14229)])),
             ('ai2d', OrderedDict([('200', 14230)])),
             ('chartqa', OrderedDict([('200', 14231)])),
             ('vqa', OrderedDict([('200', 14232)])),
             ('textvqa', OrderedDict([('200', 14233)])),
             ('infographics_w_ocr', OrderedDict([('200', 14234)])),
             ('infographics', OrderedDict([('200', 14235)])),
             ('mmbench', OrderedDict([('200', 14236)]))])

In [35]:
eval_helper.get_eval_scores_all(OUTPUT_SFT10, eval_plan="eval_v4")

,mmmu_v2,mmmu_v1,docvqa,mathvista,ai2d,chartqa,vqa,textvqa,infographics_w_ocr,infographics,mmbench
100,0.5656,0.5556,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
EVAL_CONFIG_DIR_EMB65_v5

'/fsx_0/user/tranx/adel_prod/llm_mm_aligner/experiments/aws_adel/eval_sft_babyLora_emb65_v5'

In [30]:
# train v5 - eval v5
eval_helper.run_eval_sweep(
    output_dir=OUTPUT_SFT10,
    eval_sbatch=EVAL_SBATCH,
    eval_config_dir=EVAL_CONFIG_DIR_EMB65_v5,
    aligner_parent_dir=ALIGNER_ADEL_PROD_FSDP,
    benchmarks=["mmmu"],
    eval_plan="eval_v5_bz32_ep1_train_bz32",
    checkpoint_interval=100
)

New checkpoints: [100]
Start eval for /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_flywheel_v5/checkpoint-100
eval jobs saved to: /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_flywheel_v5/eval_v5_bz32_ep1_train_bz32/eval_jobs_checkpoint-100.json


In [34]:
eval_helper.get_eval_scores_all(OUTPUT_SFT10, eval_plan="eval_v5")

,mmmu_v2,mmmu_v1,docvqa,mathvista,ai2d,chartqa,vqa,textvqa,infographics_w_ocr,infographics,mmbench


# r26a_flywheel

In [5]:
eval_helper.run_eval_sweep(
    output_dir=OUTPUT_SFT9,
    eval_sbatch=EVAL_SBATCH,
    eval_config_dir=EVAL_CONFIG_DIR_EMB65,
    aligner_parent_dir=ALIGNER_ADEL_PROD_FSDP,
    benchmarks=["mmmu"],
    eval_plan="eval_v4"
)

New checkpoints: [100, 200, 300, 400]
Start eval for /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_r26a_flywheel/checkpoint-100
eval jobs saved to: /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_r26a_flywheel/eval_v4/eval_jobs_checkpoint-100.json
Start eval for /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_r26a_flywheel/checkpoint-200
eval jobs saved to: /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_r26a_flywheel/eval_v4/eval_jobs_checkpoint-200.json
Start eval for /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_r26a_flywheel/checkpoint-300
eval jobs saved to: /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_r26a_flywheel/eval_v4/eval_jobs_checkpoint-300.json
Start eval for /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_r26a_flywheel/checkpoint-400
eval jobs saved to: /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_r26a_flywheel/eval_v4/eval_jobs_checkpoint-400.json


In [33]:
eval_helper.get_eval_scores_all(OUTPUT_SFT9, eval_plan="eval_v4")

,mmmu_v2,mmmu_v1,docvqa,mathvista,ai2d,chartqa,vqa,textvqa,infographics_w_ocr,infographics,mmbench


In [6]:
eval_helper.run_eval_sweep(
    output_dir=OUTPUT_SFT9,
    eval_sbatch=EVAL_SBATCH,
    eval_config_dir=EVAL_CONFIG_DIR_EMB65_v5,
    aligner_parent_dir=ALIGNER_ADEL_PROD_FSDP,
    benchmarks=["mmmu"],
    eval_plan="eval_v5"
)

New checkpoints: [100, 200, 300, 400]
Start eval for /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_r26a_flywheel/checkpoint-100
eval jobs saved to: /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_r26a_flywheel/eval_v5/eval_jobs_checkpoint-100.json
Start eval for /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_r26a_flywheel/checkpoint-200
eval jobs saved to: /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_r26a_flywheel/eval_v5/eval_jobs_checkpoint-200.json
Start eval for /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_r26a_flywheel/checkpoint-300
eval jobs saved to: /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_r26a_flywheel/eval_v5/eval_jobs_checkpoint-300.json
Start eval for /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_r26a_flywheel/checkpoint-400
eval jobs saved to: /fsx_0/checkpoints/tranx/MM9-SFT-70B/MH19_336px_8800_BabyLora_r26a_flywheel/eval_v5/eval_jobs_checkpoint-400.json


In [32]:
eval_helper.get_eval_scores_all(OUTPUT_SFT9, eval_plan="eval_v5")

,mmmu_v2,mmmu_v1,docvqa,mathvista,ai2d,chartqa,vqa,textvqa,infographics_w_ocr,infographics,mmbench
